In [432]:
# Import the required libraries
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')


ALGORITHM USING DMAC, FORMING TRADING SIGNALS, BACKTESTING, RISK/REWARD EVALUATION METRICS, TRADE AND PORTFOLIO EVALUATION

PART ONE: LONG POSITION TRADING ALGORITHM FOR BITCOIN


In [433]:
# Read btc.csv file into a Pandas DataFrame
# Set the date column as the DateTimeIndex
btc_df = pd.read_csv(
    Path('data/BTC_USD.csv'), 
    index_col='Date', 
    parse_dates=True, 
    infer_datetime_format=True
)

# Review the DataFrame
display(btc_df.head())
display(btc_df.tail())

,Currency,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD)
Date,,,,,
2013-09-30,BTC,123.65499,124.30466,124.75166,122.56349
2013-10-01,BTC,125.45500,123.65499,125.75850,123.63383
2013-10-02,BTC,108.58483,125.45500,125.66566,83.32833
2013-10-03,BTC,118.67466,108.58483,118.67500,107.05816
2013-10-04,BTC,121.33866,118.67466,121.93633,118.00566


,Currency,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD)
Date,,,,,
2021-07-03,BTC,34404.927059,33795.946307,34891.396542,33397.807643
2021-07-04,BTC,35601.865473,34667.931842,35888.744029,34373.758442
2021-07-05,BTC,34096.894280,35276.622227,35279.694196,33158.655426
2021-07-06,BTC,34047.547617,33699.501921,35030.941494,33556.290535
2021-07-07,BTC,34206.494528,34218.259113,35053.006360,33954.798027


CREATE TRADING SIGNALS

In [434]:
# Filter the date index and close columns
signals_df = btc_df.loc[:,["Closing Price (USD)"]]

# Review the DataFrame
signals_df.tail()

,Closing Price (USD)
Date,
2021-07-03,34404.927059
2021-07-04,35601.865473
2021-07-05,34096.894280
2021-07-06,34047.547617
2021-07-07,34206.494528


In [435]:
# Use hvplot to visualize the data
signals_df.hvplot()

:Curve   [Date]   (Closing Price (USD))

In [436]:
# Set the variables for short window and long window periods
short_window = 50
long_window = 100

In [437]:
# Generate the short and long window simple moving averages (50 and 100 days, respectively)
signals_df["SMA50"] = signals_df["Closing Price (USD)"].rolling(window=short_window).mean()
signals_df["SMA100"] = signals_df["Closing Price (USD)"].rolling(window=long_window).mean()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

,Closing Price (USD),SMA50,SMA100
Date,,,
2013-09-30,123.65499,NaN,NaN
2013-10-01,125.45500,NaN,NaN
2013-10-02,108.58483,NaN,NaN
2013-10-03,118.67466,NaN,NaN
2013-10-04,121.33866,NaN,NaN


,Closing Price (USD),SMA50,SMA100
Date,,,
2021-07-03,34404.927059,36747.523293,46691.438368
2021-07-04,35601.865473,36501.848098,46502.626565
2021-07-05,34096.894280,36271.693668,46281.251947
2021-07-06,34047.547617,36089.755195,46068.288165
2021-07-07,34206.494528,35909.964156,45834.076318


In [438]:
# Create a column to hold the trading signal
signals_df["Signal"] = 0.0

In [439]:
# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA50) greater than the long-window (SMA100)
# and 0 is when the condition is not met
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA50"][short_window:] > signals_df["SMA100"][short_window:], 1.0, 0.0
)

# Review the DataFrame
signals_df.tail(10)

,Closing Price (USD),SMA50,SMA100,Signal
Date,,,,
2021-06-28,34365.185783,38586.863990,47687.588603,0.0
2021-06-29,35945.786857,38191.468795,47469.081798,0.0
2021-06-30,34855.644515,37757.110575,47274.344657,0.0
2021-07-01,33533.501912,37384.824190,47061.736699,0.0
2021-07-02,33526.853335,37060.078615,46869.127777,0.0
2021-07-03,34404.927059,36747.523293,46691.438368,0.0
2021-07-04,35601.865473,36501.848098,46502.626565,0.0
2021-07-05,34096.894280,36271.693668,46281.251947,0.0
2021-07-06,34047.547617,36089.755195,46068.288165,0.0


In [440]:
# Slice the DataFrame to confirm the Signal
signals_df.loc[:"2021-07-21"]

,Closing Price (USD),SMA50,SMA100,Signal
Date,,,,
2013-09-30,123.654990,NaN,NaN,0.0
2013-10-01,125.455000,NaN,NaN,0.0
2013-10-02,108.584830,NaN,NaN,0.0
2013-10-03,118.674660,NaN,NaN,0.0
2013-10-04,121.338660,NaN,NaN,0.0
...,...,...,...,...
2021-07-03,34404.927059,36747.523293,46691.438368,0.0
2021-07-04,35601.865473,36501.848098,46502.626565,0.0
2021-07-05,34096.894280,36271.693668,46281.251947,0.0


In [441]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

# Review the DataFrame
signals_df.loc[:"2021-07-21"]

,Closing Price (USD),SMA50,SMA100,Signal,Entry/Exit
Date,,,,,
2013-09-30,123.654990,NaN,NaN,0.0,NaN
2013-10-01,125.455000,NaN,NaN,0.0,0.0
2013-10-02,108.584830,NaN,NaN,0.0,0.0
2013-10-03,118.674660,NaN,NaN,0.0,0.0
2013-10-04,121.338660,NaN,NaN,0.0,0.0
...,...,...,...,...,...
2021-07-03,34404.927059,36747.523293,46691.438368,0.0,0.0
2021-07-04,35601.865473,36501.848098,46502.626565,0.0,0.0
2021-07-05,34096.894280,36271.693668,46281.251947,0.0,0.0


In [442]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]["Closing Price (USD)"].hvplot.scatter(
    color='yellow',
    marker='v',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Show the plot
exit

:Scatter   [Date]   (Closing Price (USD))

In [443]:

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]["Closing Price (USD)"].hvplot.scatter(
    color='purple',
    marker='^',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Show the plot
entry



:Scatter   [Date]   (Closing Price (USD))

In [444]:
# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400)

# Show the plot
moving_avgs

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [445]:

# Visualize close price for the investment
security_close = signals_df[["Closing Price (USD)"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400)

# Show the plot
security_close



:Curve   [Date]   (Closing Price (USD))

In [446]:
# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot
entry_exit_plot.opts(
    title="Bitcoin - SMA50, SMA100, Entry and Exit Points"
)

:Overlay
   .Curve.I                                                         :Curve   [Date]   (Closing Price (USD))
   .NdOverlay.I                                                     :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Closing_Price_left_parenthesis_USD_right_parenthesis.I  :Scatter   [Date]   (Closing Price (USD))
   .Scatter.Closing_Price_left_parenthesis_USD_right_parenthesis.II :Scatter   [Date]   (Closing Price (USD))

In [447]:

#Set the intial capital to 100,000.00 USD

# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 3

In [448]:
# Buy a 3 share position when the dual moving average crossover Signal equals 1 (SMA50 is greater than SMA100)
# Sell a 3 share position when the dual moving average crossover Signal equals 0 (SMA50 is less than SMA100)
signals_df['Position'] = share_size * signals_df['Signal']

# Determine the points in time where a 3 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

#The “Portfolio Holdings” column: This column contains the value of the BTC shares that the algorithm holds.

# Multiply the close price by the number of shares held, or the Position
signals_df['Portfolio Holdings'] = signals_df['Closing Price (USD)'] * signals_df['Position']

#The “Portfolio Cash” column: This column contains the amount of available cash that the portfolio holds. The first value in this column is the value of the initial capital investment—100,000. 
# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested

signals_df['Portfolio Cash'] = initial_capital - (signals_df['Closing Price (USD)'] * signals_df['Entry/Exit Position']).cumsum()


#The “Portfolio Total” column: This column contains the current value of the portfolio, which is the value of the held stock plus the value of the portfolio cash.
# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

#The “Portfolio Cumulative Returns” column: This column calculates the cumulative return value of the portfolio, or the aggregate amount that the portfolio has gained or lost over the period
# Calculate the portfolio cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

# Print the DataFrame
signals_df.tail(150)

,Closing Price (USD),SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,
2021-02-08,44716.685469,33285.031172,25623.180839,1.0,0.0,3.0,0.0,134150.056407,96506.245481,230656.301889,0.088560,1.306563
2021-02-09,46674.851688,33754.982785,25952.627382,1.0,0.0,3.0,0.0,140024.555064,96506.245481,236530.800546,0.025469,1.365308
2021-02-10,45237.475689,34191.052684,26268.665157,1.0,0.0,3.0,0.0,135712.427068,96506.245481,232218.672549,-0.018231,1.322187
2021-02-11,47500.897524,34676.581552,26605.344748,1.0,0.0,3.0,0.0,142502.692573,96506.245481,239008.938054,0.029241,1.390089
2021-02-12,47884.182862,35161.787498,26942.546809,1.0,0.0,3.0,0.0,143652.548587,96506.245481,240158.794068,0.004811,1.401588
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-03,34404.927059,36747.523293,46691.438368,0.0,0.0,0.0,0.0,0.000000,210763.613563,210763.613563,0.000000,1.107636
2021-07-04,35601.865473,36501.848098,46502.626565,0.0,0.0,0.0,0.0,0.000000,210763.613563,210763.613563,0.000000,1.107636
2021-07-05,34096.894280,36271.693668,46281.251947,0.0,0.0,0.0,0.0,0.000000,210763.613563,210763.613563,0.000000,1.107636


Interpret Backtesting Results


In [449]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='yellow',
    marker='v',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='purple',
    marker='^',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize the value of the total portfolio
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1000,
    height=400
)

# Overlay the plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="BTC Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)


:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

ANALYSIS ON THE LONG TERM POSITION AND BACKTESTING:

First, it highlights the fact that the total value of the portfolio changed with each entry and exit of a 3-share position in BTC

Second, the trading algorithm doubled the initial investment up until the last exit signal. Before this, the price of Bitcoin went through volatility. After May 24, 2021 the price of Bitcoin continuously went down, so if the trader did not sell at the exit point, the profit would be less.

With this information, we might not want to use this algorithm—or buy this stock—for the portfolio of an investor who is risk averse or has a short-term time horizon.



NEXT: Portfolio-Level Risk/Reward Evaluation Metrics

In [450]:
signals_df.tail(10)

,Closing Price (USD),SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,
2021-06-28,34365.185783,38586.863990,47687.588603,0.0,0.0,0.0,0.0,0.0,210763.613563,210763.613563,0.0,1.107636
2021-06-29,35945.786857,38191.468795,47469.081798,0.0,0.0,0.0,0.0,0.0,210763.613563,210763.613563,0.0,1.107636
2021-06-30,34855.644515,37757.110575,47274.344657,0.0,0.0,0.0,0.0,0.0,210763.613563,210763.613563,0.0,1.107636
2021-07-01,33533.501912,37384.824190,47061.736699,0.0,0.0,0.0,0.0,0.0,210763.613563,210763.613563,0.0,1.107636
2021-07-02,33526.853335,37060.078615,46869.127777,0.0,0.0,0.0,0.0,0.0,210763.613563,210763.613563,0.0,1.107636
2021-07-03,34404.927059,36747.523293,46691.438368,0.0,0.0,0.0,0.0,0.0,210763.613563,210763.613563,0.0,1.107636
2021-07-04,35601.865473,36501.848098,46502.626565,0.0,0.0,0.0,0.0,0.0,210763.613563,210763.613563,0.0,1.107636
2021-07-05,34096.894280,36271.693668,46281.251947,0.0,0.0,0.0,0.0,0.0,210763.613563,210763.613563,0.0,1.107636
2021-07-06,34047.547617,36089.755195,46068.288165,0.0,0.0,0.0,0.0,0.0,210763.613563,210763.613563,0.0,1.107636


In [451]:
# Create a list for the column name
columns = ['Backtest']

# Create a list holding the names of the new evaluation metrics
metrics = [
    'Annualized Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

# Initialize the DataFrame with index set to the evaluation metrics and the column
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

# Review the DataFrame
portfolio_evaluation_df

,Backtest
Annualized Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [452]:
# Calculate annualized return
portfolio_evaluation_df.loc['Annualized Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)
portfolio_evaluation_df


,Backtest
Annualized Return,0.0743602
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [453]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]
portfolio_evaluation_df

,Backtest
Annualized Return,0.0743602
Cumulative Returns,1.10764
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [454]:
# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annualized Return,0.0743602
Cumulative Returns,1.10764
Annual Volatility,0.127474
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [455]:
# Calculate Sharpe ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annualized Return,0.0743602
Cumulative Returns,1.10764
Annual Volatility,0.127474
Sharpe Ratio,0.583335
Sortino Ratio,NaN


In [456]:
# Calculate downside return values

# Create a DataFrame that contains the Portfolio Daily Returns column
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()

# Create a column to hold downside return values
sortino_ratio_df.loc[:,'Downside Returns'] = 0

# Find Portfolio Daily Returns values less than 0,
# square those values, and add them to the Downside Returns column
sortino_ratio_df.loc[sortino_ratio_df['Portfolio Daily Returns'] < 0,
                     'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

# Review the DataFrame
sortino_ratio_df.tail()

,Portfolio Daily Returns,Downside Returns
Date,,
2021-07-03,0.0,0.0
2021-07-04,0.0,0.0
2021-07-05,0.0,0.0
2021-07-06,0.0,0.0
2021-07-07,0.0,0.0


In [457]:
# Calculate the Sortino ratio

# Calculate the annualized return value
annualized_return = (
    sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
)

# Calculate the annualized downside standard deviation value
downside_standard_deviation = (
    np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
)

# The Sortino ratio is reached by dividing the annualized return value
# by the downside standard deviation value
sortino_ratio = annualized_return/downside_standard_deviation

# Add the Sortino ratio to the evaluation DataFrame
portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df

,Backtest
Annualized Return,0.0743602
Cumulative Returns,1.10764
Annual Volatility,0.127474
Sharpe Ratio,0.583335
Sortino Ratio,0.873757


ANALYSIS:

Annualized Return: 7.4% means that the dollar value of the portfolio should increase by 7.8% each year

Cumulative returns: The  dollar value of the portfolio increased by 110%

Annual volatility: The annual volatility of the portfolio should have a spread of about 12.74% surrounding the annualized return. This means that the portfolio might return as much as 20.17% (7.43% + 12.74%) or lose as much as −5.31% (7.43% − 12.74%) per year.

Sharpe ratio: The Sharpe ratio, which evaluates the performance of a portfolio on a risk-adjusted basis, is 0.58. The closest the score to 1.0 the better risk/reward profile 

Sortino ratio: The Sortino ratio of our portfolio suggests a rate of about .87 which is closer to 1.0  than th Sharpe Ratio.

This type of invested is best suited for a trader or investor who is risk loving, does not mind to keep the capital in investments long term for generate a higher return.


In [458]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock',
        'Entry Date',
        'Exit Date',
        'Shares',
        'Entry Share Price',
        'Exit Share Price',
        'Entry Portfolio Holding',
        'Exit Portfolio Holding',
        'Profit/Loss']
)

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Portfolio Holdings']
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Closing Price (USD)']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Closing Price (USD)'] * row['Entry/Exit Position'])
        exit_share_price = row['Closing Price (USD)']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'BTC',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,BTC,2014-01-07,2014-02-23,3.0,855.759330,572.767500,2567.277990,1718.302500,-848.975490
1,BTC,2014-06-14,2014-08-22,3.0,570.288330,507.510000,1710.864990,1522.530000,-188.334990
2,BTC,2015-04-01,2015-05-01,3.0,246.960000,233.851100,740.880000,701.553300,-39.326700
3,BTC,2015-07-01,2015-09-10,3.0,259.629370,240.563660,778.888110,721.690980,-57.197130
4,BTC,2015-10-31,2016-02-24,3.0,311.702740,422.617000,935.108220,1267.851000,332.742780
5,BTC,2016-03-27,2016-08-29,3.0,424.297000,571.941250,1272.891000,1715.823750,442.932750
6,BTC,2016-10-19,2018-02-14,3.0,626.368750,9659.287500,1879.106250,28977.862500,27098.756250
7,BTC,2018-06-07,2018-06-16,3.0,7684.930000,6529.544015,23054.790000,19588.632045,-3466.157955
8,BTC,2018-08-27,2018-09-16,3.0,6756.628926,6281.409356,20269.886778,18844.228067,-1425.658711
9,BTC,2019-03-13,2019-09-20,3.0,3846.274442,10138.335205,11538.823325,30415.005616,18876.182291


ANALYSIS: Overall, it seems that our DMAC Long Position Trading Algorithm produces minimal loss and impressive returns on investment.

PART TWO: SHORT POSITION TRADE ALGORITHM

In [459]:
signals_shortposition_df = btc_df.loc[:,["Closing Price (USD)"]]

# Review the DataFrame
signals_shortposition_df.tail()

,Closing Price (USD)
Date,
2021-07-03,34404.927059
2021-07-04,35601.865473
2021-07-05,34096.894280
2021-07-06,34047.547617
2021-07-07,34206.494528


In [460]:
#plot the datatframe

signals_shortposition_df.hvplot()
    
    

:Curve   [Date]   (Closing Price (USD))

In [461]:
# Set the short_window (50) and long window (100) variables
short_window = 50
long_window = 100



In [462]:
# Generate the short and long moving averages (50 and 100 days, respectively)
signals_shortposition_df['SMA50'] = signals_shortposition_df["Closing Price (USD)"].rolling(window=short_window).mean()
signals_shortposition_df['SMA100'] = signals_shortposition_df["Closing Price (USD)"].rolling(window=long_window).mean()

# Review the DataFrame
signals_shortposition_df.tail()

,Closing Price (USD),SMA50,SMA100
Date,,,
2021-07-03,34404.927059,36747.523293,46691.438368
2021-07-04,35601.865473,36501.848098,46502.626565
2021-07-05,34096.894280,36271.693668,46281.251947
2021-07-06,34047.547617,36089.755195,46068.288165
2021-07-07,34206.494528,35909.964156,45834.076318


In [463]:
# Initialize the new Signal column to hold the trading signal
signals_shortposition_df['Signal'] = 0.0

In [464]:
# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA50) is less than the long-window (SMA100)
signals_shortposition_df["Signal"][short_window:] = np.where(
    signals_shortposition_df["SMA50"][short_window:] < signals_shortposition_df["SMA100"][short_window:], 1.0, 0.0
)

# Review the DataFrame
signals_shortposition_df.loc[:"2021-07-07"]

,Closing Price (USD),SMA50,SMA100,Signal
Date,,,,
2013-09-30,123.654990,NaN,NaN,0.0
2013-10-01,125.455000,NaN,NaN,0.0
2013-10-02,108.584830,NaN,NaN,0.0
2013-10-03,118.674660,NaN,NaN,0.0
2013-10-04,121.338660,NaN,NaN,0.0
...,...,...,...,...
2021-07-03,34404.927059,36747.523293,46691.438368,1.0
2021-07-04,35601.865473,36501.848098,46502.626565,1.0
2021-07-05,34096.894280,36271.693668,46281.251947,1.0


In [465]:
# Calculate the points in time at which a position should be taken, 1 or -1
signals_shortposition_df['Entry/Exit'] = signals_shortposition_df['Signal'].diff()

# Review the DataFrame
signals_shortposition_df.loc[:"2021-07-07"]


,Closing Price (USD),SMA50,SMA100,Signal,Entry/Exit
Date,,,,,
2013-09-30,123.654990,NaN,NaN,0.0,NaN
2013-10-01,125.455000,NaN,NaN,0.0,0.0
2013-10-02,108.584830,NaN,NaN,0.0,0.0
2013-10-03,118.674660,NaN,NaN,0.0,0.0
2013-10-04,121.338660,NaN,NaN,0.0,0.0
...,...,...,...,...,...
2021-07-03,34404.927059,36747.523293,46691.438368,1.0,0.0
2021-07-04,35601.865473,36501.848098,46502.626565,1.0,0.0
2021-07-05,34096.894280,36271.693668,46281.251947,1.0,0.0


Plot the Entry and Exit Points of the Trading Signal

Using hvPlot, we create two scatter plots: one for the entry points and another for the exit points. Green markers indicate the entry points (when the algorithm sells the stock). Orange markers to indicate the exit points (when the algorithm buys the stock to cover the short position).


In [466]:
# Visualize entry positions relative to close price
entry_shortposition = signals_shortposition_df[signals_shortposition_df['Entry/Exit'] == 1.0]["Closing Price (USD)"].hvplot.scatter(
    color='green',
    marker='^',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

entry_shortposition




:Scatter   [Date]   (Closing Price (USD))

In [467]:
# Visualize exit positions relative to close price
exit_shortposition = signals_shortposition_df[signals_shortposition_df['Entry/Exit'] == -1.0]["Closing Price (USD)"].hvplot.scatter(
    color='orange',
    marker='v',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

exit_shortposition

:Scatter   [Date]   (Closing Price (USD))

In [468]:
# Plot the entry and exit points
entry_shortposition * exit_shortposition

:Overlay
   .Scatter.Closing_Price_left_parenthesis_USD_right_parenthesis.I  :Scatter   [Date]   (Closing Price (USD))
   .Scatter.Closing_Price_left_parenthesis_USD_right_parenthesis.II :Scatter   [Date]   (Closing Price (USD))

In [469]:
# Visualize Close price for the bitcoin
security_close_shortterm = signals_shortposition_df[["Closing Price (USD)"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400)

# Visualize the SMA50 and SMA 100 moving averages
moving_avgs_short = signals_shortposition_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400)

In [470]:
# Plot the security_close and moving_avgs
security_close_shortterm * moving_avgs_short

:Overlay
   .Curve.I     :Curve   [Date]   (Closing Price (USD))
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [Date]   (value)

In [471]:
# Overlay all four plots in a single visualization
entry_exit_plot = security_close_shortterm * moving_avgs_short * entry_shortposition * exit_shortposition
entry_exit_plot.opts(
    title="BITCOIN - Short-Position Dual Moving Average Trading Algorithm"
)

:Overlay
   .Curve.I                                                         :Curve   [Date]   (Closing Price (USD))
   .NdOverlay.I                                                     :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Closing_Price_left_parenthesis_USD_right_parenthesis.I  :Scatter   [Date]   (Closing Price (USD))
   .Scatter.Closing_Price_left_parenthesis_USD_right_parenthesis.II :Scatter   [Date]   (Closing Price (USD))



For the short position, it shows that a most recent good entry was on 07/02/2021 but did not create a exit trading signal yet. The price of Bitcoin has continuously dropped since then.


BACKTEST SHORT POSITION TRADING ALGORITHM

In [472]:
# Set the initial_capital to 100000
initial_capital = float(100000)


Set a share_size variable to negative 3 shares (-3).

In [473]:
# Set the share_size to -3
# In a short-position strategy, shares are sold before they are bought 
share_size = -3

Create a new column named “Position” by multiplying the share_size value by the values in the “Signal” column.

In [474]:
# Create a column named "Position" by multiplying the share_size by the Signal
# Sell a position (-3shares) when the dual moving average crossover Signal equals 1 (SMA50 is less than SMA100)
# Buy a position (3 shares) when the dual moving average crossover Signal equals 0 (SMA50 is greater than SMA100)
signals_shortposition_df["Short Position"] = share_size * signals_shortposition_df["Signal"]

# Review the DataFrame
signals_shortposition_df.tail()

,Closing Price (USD),SMA50,SMA100,Signal,Entry/Exit,Short Position
Date,,,,,,
2021-07-03,34404.927059,36747.523293,46691.438368,1.0,0.0,-3.0
2021-07-04,35601.865473,36501.848098,46502.626565,1.0,0.0,-3.0
2021-07-05,34096.894280,36271.693668,46281.251947,1.0,0.0,-3.0
2021-07-06,34047.547617,36089.755195,46068.288165,1.0,0.0,-3.0
2021-07-07,34206.494528,35909.964156,45834.076318,1.0,0.0,-3.0


In [475]:
# Create a Portfolio Holdings column by multiplying the Close price by the Position
signals_shortposition_df["Portfolio Holdings"] = signals_shortposition_df["Closing Price (USD)"] * signals_shortposition_df["Short Position"]


# Review the DataFrame
signals_shortposition_df.tail()

,Closing Price (USD),SMA50,SMA100,Signal,Entry/Exit,Short Position,Portfolio Holdings
Date,,,,,,,
2021-07-03,34404.927059,36747.523293,46691.438368,1.0,0.0,-3.0,-103214.781178
2021-07-04,35601.865473,36501.848098,46502.626565,1.0,0.0,-3.0,-106805.596419
2021-07-05,34096.894280,36271.693668,46281.251947,1.0,0.0,-3.0,-102290.682840
2021-07-06,34047.547617,36089.755195,46068.288165,1.0,0.0,-3.0,-102142.642852
2021-07-07,34206.494528,35909.964156,45834.076318,1.0,0.0,-3.0,-102619.483585


In [476]:
# To calculate Portfolio Cash, subtract the cumulative sum of the trade cost/proceeds from the initial_capital
# The trade cost proceeds are calculated by multiplying the Close price by Entry/Exit Position
signals_shortposition_df["Portfolio Cash"] = (
    initial_capital - (signals_shortposition_df["Closing Price (USD)"] * signals_df["Entry/Exit"]).cumsum()
)

# Review the DataFrame
signals_shortposition_df.tail()

,Closing Price (USD),SMA50,SMA100,Signal,Entry/Exit,Short Position,Portfolio Holdings,Portfolio Cash
Date,,,,,,,,
2021-07-03,34404.927059,36747.523293,46691.438368,1.0,0.0,-3.0,-103214.781178,136921.204521
2021-07-04,35601.865473,36501.848098,46502.626565,1.0,0.0,-3.0,-106805.596419,136921.204521
2021-07-05,34096.894280,36271.693668,46281.251947,1.0,0.0,-3.0,-102290.682840,136921.204521
2021-07-06,34047.547617,36089.755195,46068.288165,1.0,0.0,-3.0,-102142.642852,136921.204521
2021-07-07,34206.494528,35909.964156,45834.076318,1.0,0.0,-3.0,-102619.483585,136921.204521


In [477]:
# Calculate the Portfolio Total value by adding Portfolio Cash and Portfolio Holdings 
signals_shortposition_df["Portfolio Total"] = signals_shortposition_df["Portfolio Cash"] + signals_df["Portfolio Holdings"]

# Review the DataFrame
signals_shortposition_df.tail()

,Closing Price (USD),SMA50,SMA100,Signal,Entry/Exit,Short Position,Portfolio Holdings,Portfolio Cash,Portfolio Total
Date,,,,,,,,,
2021-07-03,34404.927059,36747.523293,46691.438368,1.0,0.0,-3.0,-103214.781178,136921.204521,136921.204521
2021-07-04,35601.865473,36501.848098,46502.626565,1.0,0.0,-3.0,-106805.596419,136921.204521,136921.204521
2021-07-05,34096.894280,36271.693668,46281.251947,1.0,0.0,-3.0,-102290.682840,136921.204521,136921.204521
2021-07-06,34047.547617,36089.755195,46068.288165,1.0,0.0,-3.0,-102142.642852,136921.204521,136921.204521
2021-07-07,34206.494528,35909.964156,45834.076318,1.0,0.0,-3.0,-102619.483585,136921.204521,136921.204521


In [478]:
# Calculate the Portfolio Daily Returns based on the Portfolio Total
signals_shortposition_df["Portfolio Daily Returns"] = signals_shortposition_df["Portfolio Total"].pct_change()

# Review the DataFrame
signals_shortposition_df.tail()

,Closing Price (USD),SMA50,SMA100,Signal,Entry/Exit,Short Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns
Date,,,,,,,,,,
2021-07-03,34404.927059,36747.523293,46691.438368,1.0,0.0,-3.0,-103214.781178,136921.204521,136921.204521,0.0
2021-07-04,35601.865473,36501.848098,46502.626565,1.0,0.0,-3.0,-106805.596419,136921.204521,136921.204521,0.0
2021-07-05,34096.894280,36271.693668,46281.251947,1.0,0.0,-3.0,-102290.682840,136921.204521,136921.204521,0.0
2021-07-06,34047.547617,36089.755195,46068.288165,1.0,0.0,-3.0,-102142.642852,136921.204521,136921.204521,0.0
2021-07-07,34206.494528,35909.964156,45834.076318,1.0,0.0,-3.0,-102619.483585,136921.204521,136921.204521,0.0


In [479]:
# Calculate the Portfolio Cumulative Returns based on the Portfolio Daily Returns
signals_shortposition_df["Portfolio Cumulative Returns"] = (
    1 + signals_shortposition_df["Portfolio Daily Returns"]
).cumprod() - 1

# Review the DataFrame
signals_shortposition_df.tail()

,Closing Price (USD),SMA50,SMA100,Signal,Entry/Exit,Short Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,
2021-07-03,34404.927059,36747.523293,46691.438368,1.0,0.0,-3.0,-103214.781178,136921.204521,136921.204521,0.0,0.369212
2021-07-04,35601.865473,36501.848098,46502.626565,1.0,0.0,-3.0,-106805.596419,136921.204521,136921.204521,0.0,0.369212
2021-07-05,34096.894280,36271.693668,46281.251947,1.0,0.0,-3.0,-102290.682840,136921.204521,136921.204521,0.0,0.369212
2021-07-06,34047.547617,36089.755195,46068.288165,1.0,0.0,-3.0,-102142.642852,136921.204521,136921.204521,0.0,0.369212
2021-07-07,34206.494528,35909.964156,45834.076318,1.0,0.0,-3.0,-102619.483585,136921.204521,136921.204521,0.0,0.369212


In [480]:
# Visualize the entry positions relative to the Portfolio Total
entry = signals_df[signals_df["Entry/Exit"] == 1.0]["Portfolio Total"].hvplot.scatter(
    color='green',
    marker='^',
    legend=False, 
    ylabel="Total Portfolio Value", 
    width=1000, 
    height=400
)

# Visualize the exit positions relative to the Portfolio Total
exit = signals_df[signals_df["Entry/Exit"] == -1.0]["Portfolio Total"].hvplot.scatter(
    color='orange',
    marker='v',
    legend=False, 
    ylabel="Total Portfolio Value", 
    width=1000, 
    height=400
)

# Visualize Portfolio Total for the investment
total_portfolio_value = signals_df[["Portfolio Total"]].hvplot(
    line_color="lightgray", 
    ylabel="Total Portfolio Value", 
    width=1000, 
    height=400
)

# Overlay the entry, exit and total_portfolio_value plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="BTC Short-Position Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

Based on this information gathered through backtesting short position trading algorithm,
the trader made some money when it recommended to buy low and sell high. Let's dig deeper:

In [481]:
shortposition_metrics = [
    'Annualized Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio'
]

# Create a list that holds the column name
columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and columns 
shortportfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

# Review the DataFrame
shortportfolio_evaluation_df

,Backtest
Annualized Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [482]:
# Calculate the Annualized return metric
shortportfolio_evaluation_df.loc['Annualized Return'] = (
    signals_shortposition_df['Portfolio Daily Returns'].mean() * 252)

shortportfolio_evaluation_df



,Backtest
Annualized Return,0.0529698
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [483]:
# Calculate the Cumulative returns metric
shortportfolio_evaluation_df.loc['Cumulative Returns'] = signals_shortposition_df['Portfolio Cumulative Returns'][-1]

shortportfolio_evaluation_df

,Backtest
Annualized Return,0.0529698
Cumulative Returns,0.369212
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [484]:
# Calculate the Annual volatility metric
shortportfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_shortposition_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

shortportfolio_evaluation_df

,Backtest
Annualized Return,0.0529698
Cumulative Returns,0.369212
Annual Volatility,0.2193
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [485]:
# Calculate the Sharpe ratio
shortportfolio_evaluation_df.loc['Sharpe Ratio'] = (
     signals_shortposition_df['Portfolio Daily Returns'].mean() * 252) / (
     signals_shortposition_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

shortportfolio_evaluation_df

,Backtest
Annualized Return,0.0529698
Cumulative Returns,0.369212
Annual Volatility,0.2193
Sharpe Ratio,0.24154
Sortino Ratio,NaN


In [486]:
# Calculate the Sortino ratio
# Start by calculating the downside return values

# Create a DataFrame that contains the Portfolio Daily Returns column
sortino_ratio_df = signals_shortposition_df[['Portfolio Daily Returns']]

# Create a column to hold downside return values
sortino_ratio_df.loc[:,'Downside Returns'] = 0

# Find Portfolio Daily Returns values less than 0, 
# square those values, and add them to the Downside Returns column
sortino_ratio_df.loc[sortino_ratio_df['Portfolio Daily Returns'] < 0, 
                     'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

# Calculate the annualized return value
annualized_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252

# Calculate the annualized downside standard deviation value
downside_standard_deviation = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)

# Divide the annualized return value by the downside standard deviation value
sortino_ratio = annualized_return/downside_standard_deviation

# Add the Sortino ratio to the evaluation DataFrame
shortportfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio

shortportfolio_evaluation_df

,Backtest
Annualized Return,0.0529698
Cumulative Returns,0.369212
Annual Volatility,0.2193
Sharpe Ratio,0.24154
Sortino Ratio,0.331471


In [487]:
# Initialize the trade evaluation DataFrame
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

# Loop through the signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = abs(row['Portfolio Holdings'])
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Closing Price (USD)']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Closing Price (USD)'] * row['Entry/Exit Position'])
        exit_share_price = row['Closing Price (USD)']
        profit_loss =  entry_portfolio_holding - exit_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'BTC',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)


trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,BTC,2014-01-07,2014-02-23,3.0,855.759330,572.767500,2567.277990,1718.302500,848.975490
1,BTC,2014-06-14,2014-08-22,3.0,570.288330,507.510000,1710.864990,1522.530000,188.334990
2,BTC,2015-04-01,2015-05-01,3.0,246.960000,233.851100,740.880000,701.553300,39.326700
3,BTC,2015-07-01,2015-09-10,3.0,259.629370,240.563660,778.888110,721.690980,57.197130
4,BTC,2015-10-31,2016-02-24,3.0,311.702740,422.617000,935.108220,1267.851000,-332.742780
5,BTC,2016-03-27,2016-08-29,3.0,424.297000,571.941250,1272.891000,1715.823750,-442.932750
6,BTC,2016-10-19,2018-02-14,3.0,626.368750,9659.287500,1879.106250,28977.862500,-27098.756250
7,BTC,2018-06-07,2018-06-16,3.0,7684.930000,6529.544015,23054.790000,19588.632045,3466.157955
8,BTC,2018-08-27,2018-09-16,3.0,6756.628926,6281.409356,20269.886778,18844.228067,1425.658711
9,BTC,2019-03-13,2019-09-20,3.0,3846.274442,10138.335205,11538.823325,30415.005616,-18876.182291


TRADE EVALUATION FOR SHORT POSITION ON BITCOIN: The first four trades returned a small gain, but half of the  short trades ended in a sizable loss of funds. Overall, the trader lost a lot of money over time. The current market is on a downtrend, as BTC has been consolidating/trading sideways for a more than two months now and therefore there is not a good opportunity to sell high yet from the last recommended entry. This trading strategy would not work for risk averse trader/investor who has a short time horizon.